In [1]:
from langchain.document_loaders import GitLoader
from langchain.vectorstores import Chroma
# Initialize the GitLoader
loader = GitLoader(
    repo_path="./data/",
    clone_url="https://github.com/taradepan/VoiceGPT",
    branch="main",
)

# Load the data from the GitHub repository
data = loader.load()

# Print the loaded data
for doc in data:
    print(doc)
    print("\n\n")

page_content='.env\n__pycache__/\ninput.wav\noutput.mp3' metadata={'source': '.gitignore', 'file_path': '.gitignore', 'file_name': '.gitignore', 'file_type': ''}



page_content='import pyaudio\nimport wave\nimport time\n\ndef record_audio(output_file):\n    audio = pyaudio.PyAudio()\n    stream = audio.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=1024)\n    frames = []\n\n    try:\n        print("Recording...")\n        start_time = time.time()\n\n        while time.time() - start_time < 4:\n            data = stream.read(1024)\n            frames.append(data)\n    except KeyboardInterrupt:\n        print("Recording stopped.")\n        pass\n\n    stream.stop_stream()\n    stream.close()\n    audio.terminate()\n\n    waveFile = wave.open(output_file, "wb")\n    waveFile.setnchannels(1)\n    waveFile.setsampwidth(audio.get_sample_size(pyaudio.paInt16))\n    waveFile.setframerate(16000)\n    waveFile.writeframes(b\'\'.join(frames))\n    waveFile.clo

In [2]:
print(data[1].page_content)

import pyaudio
import wave
import time

def record_audio(output_file):
    audio = pyaudio.PyAudio()
    stream = audio.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=1024)
    frames = []

    try:
        print("Recording...")
        start_time = time.time()

        while time.time() - start_time < 4:
            data = stream.read(1024)
            frames.append(data)
    except KeyboardInterrupt:
        print("Recording stopped.")
        pass

    stream.stop_stream()
    stream.close()
    audio.terminate()

    waveFile = wave.open(output_file, "wb")
    waveFile.setnchannels(1)
    waveFile.setsampwidth(audio.get_sample_size(pyaudio.paInt16))
    waveFile.setframerate(16000)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()

if __name__ == "__main__":
    record_audio("output.wav")  


In [3]:
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key="HUGGING_FACE_TOKEN", model_name="sentence-transformers/all-MiniLM-l6-v2"
)

db = Chroma.from_documents(data, embeddings)

In [4]:
query = "What is name of the project?"
docs = db.similarity_search(query)
print(docs)

[Document(page_content='**VoiceGPT**\n\nThis is an open source sample project.\nTo get started with Voice GPT:\n1. Download this repo.\n2. Install the Required Packages.\n3. Create `.env` file and add your OpenAI and ElevenLabs API Key.\n4. Run the `main.py` file.\n\nFeel free to use this repo. But make sure to give this repo a Star. And this repo is also open to contributions.\n', metadata={'file_name': 'readme.md', 'file_path': 'readme.md', 'file_type': '.md', 'source': 'readme.md'}), Document(page_content='openai\npyaudio\nwave\ndotenv\npygame\nmutagen', metadata={'file_name': 'requirements.txt', 'file_path': 'requirements.txt', 'file_type': '.txt', 'source': 'requirements.txt'}), Document(page_content='from dotenv import load_dotenv\nload_dotenv()\nimport os\nimport pygame\nimport openai\nfrom voice import voice\nfrom test import play_audio\nimport mutagen.mp3\nopenai.api_key = os.getenv("OPENAI_API_KEY")\n\ndef chat(userInput):\n    completion = openai.ChatCompletion.create(\n    

In [5]:
from langchain.chains import RetrievalQA
from langchain import PromptTemplate
from langchain.llms import HuggingFaceHub

repo_id = "codellama/CodeLlama-34b-Instruct-hf"
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.1, "max_length": 200}, huggingfacehub_api_token="HUGGING_FACE_TOKEN"
)

/home/taradepan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/taradepan/.local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [6]:
prompt_template = """Use the following pieces of context to answer the question enclosed within  3 backticks at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Please provide an answer which is factually correct and based on the information retrieved from the vector store.
Please also mention any quotes supporting the answer if any present in the context supplied within two double quotes "" .

{context}

QUESTION:```{question}```
ANSWER:
"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context","question"]
  )

In [7]:
qa = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  chain_type_kwargs={"prompt": PROMPT},
                                  retriever=db.as_retriever(search_kwargs={"k": 5}),
                                  return_source_documents=True
                                  )

questions = input("User :")
print(questions)
result = qa(questions)
#
print(result.keys())


dict_keys(['query', 'result', 'source_documents'])


In [8]:
print(result["source_documents"])

[Document(page_content='openai\npyaudio\nwave\ndotenv\npygame\nmutagen', metadata={'file_name': 'requirements.txt', 'file_path': 'requirements.txt', 'file_type': '.txt', 'source': 'requirements.txt'}), Document(page_content='from dotenv import load_dotenv\nload_dotenv()\nimport os\nimport pygame\nimport openai\nfrom voice import voice\nfrom test import play_audio\nimport mutagen.mp3\nopenai.api_key = os.getenv("OPENAI_API_KEY")\n\ndef chat(userInput):\n    completion = openai.ChatCompletion.create(\n    model="gpt-3.5-turbo",\n    max_tokens=100,\n    messages=[\n        {"role": "system", "content": "You are a personal A.I. assistant and your name is \'Bella\'. try to answer the user\'s questions under 100 tokens"},\n        {"role": "user", "content": userInput},\n    ]\n    )\n\n    print(completion.choices[0].message.content)\n\n    voice(completion.choices[0].message.content)\n    play_audio("output.mp3")\n\n# chat("Hello, what is openai?")\n\ndef get_audio_duration(audio_file):\n

In [9]:
eval_questions = [
    "What is the name of the repo?",
    "Which OpenAI model is used?",
    "list the required libries?",
    "what are the gitingnore files?",
    "How to setup the project?",
]

eval_answers = [
    "VoiceGPT",  
    "gpt-3.5-turbo has been used for chat completion and Wisper-1 has been used for transcribing the audio input.",
    """the required libraries are: 
    openai
    pyaudio
    wave
    dotenv
    pygame
    mutagen""",
    """ .env
    __pycache__/
    input.wav
    output.mp3""",
    """ To get started with Voice GPT:
    1. Download this repo.
    2. Install the Required Packages.
    3. Create `.env` file and add your OpenAI and ElevenLabs API Key.
    4. Run the `main.py` file."""


]

examples = [
    {"query": q, "ground_truths": [eval_answers[i]]}
    for i, q in enumerate(eval_questions)]
print(examples)


[{'query': 'What is the name of the repo?', 'ground_truths': ['VoiceGPT']}, {'query': 'Which OpenAI model is used?', 'ground_truths': ['gpt-3.5-turbo has been used for chat completion and Wisper-1 has been used for transcribing the audio input.']}, {'query': 'list the required libries?', 'ground_truths': ['the required libraries are: \n    openai\n    pyaudio\n    wave\n    dotenv\n    pygame\n    mutagen']}, {'query': 'what are the gitingnore files?', 'ground_truths': [' .env\n    __pycache__/\n    input.wav\n    output.mp3']}, {'query': 'How to setup the project?', 'ground_truths': [' To get started with Voice GPT:\n    1. Download this repo.\n    2. Install the Required Packages.\n    3. Create `.env` file and add your OpenAI and ElevenLabs API Key.\n    4. Run the `main.py` file.']}]


In [1]:
import os

os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"
# print(os.environ.get('OPENAI_API_KEY'))

In [12]:
from ragas.langchain.evalchain import RagasEvaluatorChain
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
)

faithfulness_chain = RagasEvaluatorChain(metric=faithfulness)
answer_rel_chain = RagasEvaluatorChain(metric=answer_relevancy)
context_rel_chain = RagasEvaluatorChain(metric=context_relevancy)
context_recall_chain = RagasEvaluatorChain(metric=context_recall)

In [13]:
result = qa(examples[0])
print(result["result"])


VoiceGPT

































































































In [14]:
eval_result = faithfulness_chain(result)
print(eval_result["faithfulness_score"])

1.0


In [15]:
eval_result = context_recall_chain(result)
print(eval_result["context_recall_score"])

1.0


In [16]:
eval_result = answer_rel_chain(result)
print(eval_result['answer_relevancy_score'])

0.7221705787362578


In [18]:
eval_result = context_rel_chain(result)
print(eval_result['context_relevancy_score'])

0.0967741935483871


In [19]:
from tqdm.auto import tqdm
from uuid import uuid4
predictions = qa.batch(examples)

# evaluate faitfulness
print("evaluating...")
r = faithfulness_chain.evaluate(examples, predictions)
r

evaluating...


100%|██████████| 1/1 [00:53<00:00, 53.04s/it]


[{'faithfulness_score': 1.0},
 {'faithfulness_score': 1.0},
 {'faithfulness_score': 1.0},
 {'faithfulness_score': 0.75},
 {'faithfulness_score': 1.0}]

In [27]:
print("evaluating...")
r = context_recall_chain.evaluate(examples, predictions)
r

evaluating...


0it [00:00, ?it/s]


[]

In [21]:
# evaluate answer relevancy
print("evaluating...")
r = answer_rel_chain.evaluate(examples, predictions)
r

evaluating...


100%|██████████| 1/1 [00:14<00:00, 14.31s/it]


[{'answer_relevancy_score': 0.7221705787362578},
 {'answer_relevancy_score': 0.9204680690036255},
 {'answer_relevancy_score': 0.9170556528465212},
 {'answer_relevancy_score': 0.8026047335243285},
 {'answer_relevancy_score': 0.8098258595120184}]

In [22]:
# evaluate context relevancy
print("evaluating...")
r = context_rel_chain.evaluate(examples, predictions)
r

evaluating...


100%|██████████| 1/1 [00:09<00:00,  9.69s/it]


[{'context_relevancy_score': 0.0967741935483871},
 {'context_relevancy_score': 0.01020408163265306},
 {'context_relevancy_score': 0.061224489795918366},
 {'context_relevancy_score': 0.08064516129032258},
 {'context_relevancy_score': 0.08108108108108109}]